In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

data_path = "../data-unversioned/p1ch7"
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4823, 0.4468),
                         (0.2470, 0.2435, 0.2616))
])

cifar10_train = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)
cifar10_test  = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)

trainX = torch.stack([img for img, _ in cifar10_train]).to(device)
trainY = torch.tensor([label for _, label in cifar10_train], dtype=torch.long).to(device)

testX = torch.stack([img for img, _ in cifar10_test]).to(device)
testY = torch.tensor([label for _, label in cifar10_test], dtype=torch.long).to(device)

train_dataset = torch.utils.data.TensorDataset(trainX, trainY)
val_dataset   = torch.utils.data.TensorDataset(testX, testY)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = torch.utils.data.DataLoader(val_dataset, batch_size=1000, shuffle=False)

Using device: cuda


In [2]:
class NetRes(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(4 * 4 * (n_chans1 // 2), 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out1 = out
        out = F.max_pool2d(torch.relu(self.conv3(out)) + out1, 2)

        out = out.view(-1, 4 * 4 * (self.n_chans1 // 2))
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

model = NetRes().to(device)

In [3]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)
n_epochs = 300
start = time.time()

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0

    for batch_X, batch_Y in train_loader:
        batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = loss_fn(outputs, batch_Y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_X.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

end = time.time()
print("\nTraining Time =", end - start, "seconds")
print("Final Training Loss =", epoch_loss)

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch_X, batch_Y in val_loader:
        batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)
        outputs = model(batch_X)
        preds = outputs.argmax(dim=1)
        correct += (preds == batch_Y).sum().item()
        total += batch_Y.size(0)

accuracy = correct / total
print("Evaluation Accuracy =", accuracy)

Epoch 1, Loss: 2.2979
Epoch 2, Loss: 2.2646
Epoch 3, Loss: 2.2081
Epoch 4, Loss: 2.1296
Epoch 5, Loss: 2.0636
Epoch 6, Loss: 2.0118
Epoch 7, Loss: 1.9596
Epoch 8, Loss: 1.9011
Epoch 9, Loss: 1.8425
Epoch 10, Loss: 1.7923
Epoch 11, Loss: 1.7499
Epoch 12, Loss: 1.7119
Epoch 13, Loss: 1.6770
Epoch 14, Loss: 1.6448
Epoch 15, Loss: 1.6147
Epoch 16, Loss: 1.5857
Epoch 17, Loss: 1.5595
Epoch 18, Loss: 1.5359
Epoch 19, Loss: 1.5162
Epoch 20, Loss: 1.4983
Epoch 21, Loss: 1.4817
Epoch 22, Loss: 1.4672
Epoch 23, Loss: 1.4533
Epoch 24, Loss: 1.4398
Epoch 25, Loss: 1.4274
Epoch 26, Loss: 1.4155
Epoch 27, Loss: 1.4033
Epoch 28, Loss: 1.3913
Epoch 29, Loss: 1.3801
Epoch 30, Loss: 1.3689
Epoch 31, Loss: 1.3575
Epoch 32, Loss: 1.3467
Epoch 33, Loss: 1.3358
Epoch 34, Loss: 1.3261
Epoch 35, Loss: 1.3152
Epoch 36, Loss: 1.3054
Epoch 37, Loss: 1.2954
Epoch 38, Loss: 1.2861
Epoch 39, Loss: 1.2772
Epoch 40, Loss: 1.2671
Epoch 41, Loss: 1.2588
Epoch 42, Loss: 1.2496
Epoch 43, Loss: 1.2401
Epoch 44, Loss: 1.23